In [ ]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.colors import LinearSegmentedColormap, Normalize


from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geoviews
from geopy import distance


import sympy as sp
from linearmodels.panel import RandomEffects
import statsmodels.api as sm
from factor_analyzer import FactorAnalyzer 



#### 1. Phones format

In [ ]:

def format_israeli_phone(phone_number):
    """
    Clean and format Israeli phone numbers according to standard patterns.
    
    Args:
        phone_number: A phone number string to format
        
    Returns:
        Formatted phone number string or np.nan if invalid
    """
    p = str(phone_number)
    p = re.sub(r'[א-תa-zA-Z()!@#$%^&*_=+<>?/|\\\[\]{}.,]', '', p)
    p = p.replace("-", "").replace(" ", "").replace("+972", "").strip()
    
    if len(p) in [8, 9] and not p.startswith("0"):
        p = "0" + p
    
    if p.startswith(("1700", "1800")) and len(p) == 10:
        return f"{p[:4]}-{p[4:]}"
    elif p.startswith(("072", "073", "074", "076", "077", "078", "079")) and len(p) == 10:
        return f"{p[:3]}-{p[3:]}"
    elif len(p) == 10 and p.startswith("0"):
        return f"{p[:3]}-{p[3:]}"
    elif len(p) == 9 and p.startswith("0"):
        return f"{p[:2]}-{p[2:]}"
    else:
        return np.nan

def clean_phone_numbers(df, column_name, new_column_name='טלפון'):
    """
    Process phone numbers in a DataFrame column and add formatted numbers to a new column.
    
    Args:
        df: DataFrame containing phone numbers
        column_name: Name of the column with raw phone numbers
        new_column_name: Name for the new column with formatted numbers
        
    Returns:
        DataFrame with added formatted phone numbers column
    """
    result_df = df.copy()
    
    result_df[new_column_name] = result_df[column_name].apply(format_israeli_phone)
    
    return result_df

#### 2. ID 
- as a string - 9 digits with leading zeros

In [ ]:
def isNaN(num):
    return num != num

def id9(i):

    if i is None:
        return np.nan
    if isinstance(i, float) and np.isnan(i):
        return np.nan
    
    s = str(i).replace('\u200e', '').strip()  
    
    if s.endswith('.0'):
        s = s[:-2]
    elif s.endswith('.'):
        s = s[:-1]
    
    if not s:
        return np.nan
    
    try:
        num = int(s)
    except ValueError:
        return np.nan
    
    if num == 0:
        return np.nan
    
    s = str(num)
    
    if len(s) < 9:
        s = s.zfill(9)
    elif len(s) > 9:
        s = s[:9]
    
    return s

#### 3. Sampling Data Randomly by Conditions & Row number

In [ ]:
def sample(df):

    data_for_mimun = df.copy()
    data_for_mimun['row_number'] = range(1,len(data_for_mimun)+1)

    values = [1, 2, 3, 4]

    conditions0 = [
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'תעשייה') & (data_for_mimun['קבוצת גודל'] == 'עצמאים וללא מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'בינוי') & (data_for_mimun['קבוצת גודל'] == 'עצמאים וללא מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'מסחר') & (data_for_mimun['קבוצת גודל'] == 'עצמאים וללא מועסקים'),
        (data_for_mimun['row_number'] % 4 == 1) & (data_for_mimun['ענף'] == 'שירותים') & (data_for_mimun['קבוצת גודל'] == 'עצמאים וללא מועסקים'),
    ]

    data_for_mimun['size_plus_anaf'] = np.select(conditions0, values, default=None)

    data_for_t0 = data_for_mimun[data_for_mimun['size_plus_anaf'].notnull()]

    conditions1 = [
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'תעשייה') & (data_for_mimun['קבוצת גודל'] == '1-4 מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'בינוי') & (data_for_mimun['קבוצת גודל'] == '1-4 מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'מסחר') & (data_for_mimun['קבוצת גודל'] == '1-4 מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'שירותים') & (data_for_mimun['קבוצת גודל'] == '1-4 מועסקים'),
    ]

    data_for_mimun['size_plus_anaf'] = np.select(conditions1, values, default=None)

    data_for_t1 = data_for_mimun[data_for_mimun['size_plus_anaf'].notnull()]

    conditions2 = [
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'תעשייה') & (data_for_mimun['קבוצת גודל'] == '5-19 מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'בינוי') & (data_for_mimun['קבוצת גודל'] == '5-19 מועסקים'),
        (data_for_mimun['row_number'] % 3 == 1) & (data_for_mimun['ענף'] == 'מסחר') & (data_for_mimun['קבוצת גודל'] == '5-19 מועסקים'),
        (data_for_mimun['row_number'] % 5 == 1) & (data_for_mimun['ענף'] == 'שירותים') & (data_for_mimun['קבוצת גודל'] == '5-19 מועסקים'),
    ]

    data_for_mimun['size_plus_anaf'] = np.select(conditions2, values, default=None)

    data_for_t2 = data_for_mimun[data_for_mimun['size_plus_anaf'].notnull()]


    conditions3 = [
        (data_for_mimun['row_number'] % 4 == 1) & (data_for_mimun['ענף'] == 'תעשייה') & (data_for_mimun['קבוצת גודל'] == '20-99 מועסקים'),
        (data_for_mimun['row_number'] % 4 == 1) & (data_for_mimun['ענף'] == 'בינוי') & (data_for_mimun['קבוצת גודל'] == '20-99 מועסקים'),
        (data_for_mimun['row_number'] % 4 == 1) & (data_for_mimun['ענף'] == 'מסחר') & (data_for_mimun['קבוצת גודל'] == '20-99 מועסקים'),
        (data_for_mimun['row_number'] % 4 == 1) & (data_for_mimun['ענף'] == 'שירותים') & (data_for_mimun['קבוצת גודל'] == '20-99 מועסקים'),
    ]

    data_for_mimun['size_plus_anaf'] = np.select(conditions3, values, default=None)

    data_for_t3 = data_for_mimun[data_for_mimun['size_plus_anaf'].notnull()]

    mimun_data = pd.concat([data_for_t0,data_for_t1,data_for_t2,data_for_t3], ignore_index=True)
    
    return mimun_data

#### 4. Text to DataFrame

In [ ]:
def txt_to_df(path):
    l = []
    file = open(path, "r", encoding="utf-8")
    for i in file:
        l.append(i[:132])

#     l1 = []
#     file1 = open(path, "r")
#     for i in file1:
#         l1.append(i)

    idn = []
    family = []
    first = []
    bod = []
    year = []
    identified = []
    kind = []
    jan = []
    feb = []
    march = []
    apr = []
    may = []
    june = []
    july = []
    aug = []
    sept = []
    octob = []
    novem = []
    december = []
    wage = []
    employer = []
    file_n = []
    report = []
    branch4 = []
    for i in l:
        idn.append(i[:9])
        family.append(i[9:26][::-1])
        first.append(i[26:41][::-1])
        bod.append(i[41:49])
        year.append(i[49:53])
        identified.append(i[53:54])
        kind.append(i[54:64][::-1])
        december.append(i[64:65])
        novem.append(i[65:66])
        octob.append(i[66:67])
        sept.append(i[67:68])
        aug.append(i[68:69])
        july.append(i[69:70])
        june.append(i[70:71])
        may.append(i[71:72])
        apr.append(i[72:73])
        march.append(i[73:74])
        feb.append(i[74:75])
        jan.append(i[75:76])
        wage.append(i[76:86])
        employer.append(i[86:118][::-1])
        file_n.append(i[118:127])
        report.append(i[127:128])
        branch4.append(i[128:132])
#     report[-1] = l1[-1][-1]

    df = pd.DataFrame()
    df['ID'] = idn
    df['family_name'] = family
    df['first_name'] = first
    df['date_of_birth'] = bod
    df['year_of_wage'] = year
    df['identified'] = identified
    df['status'] = kind
    df['worked12'] = december
    df['worked11'] = novem
    df['worked10'] = octob
    df['worked9'] = sept
    df['worked8'] = aug
    df['worked7'] = july
    df['worked6'] = june
    df['worked5'] = may
    df['worked4'] = apr
    df['worked3'] = march
    df['worked2'] = feb
    df['worked1'] = jan
    df['wage'] = wage
    df['employer_name'] = employer
    df['file_num'] = file_n
    df['report'] = report
    df['branch4'] = branch4

    df['identified'] = pd.Series(np.where(df.identified.values == 'כ', 1, 0),df.index)
    df['worked12'] = pd.Series(np.where(df.worked12.values == 'כ', 1, 0),df.index)
    df['worked11'] = pd.Series(np.where(df.worked11.values == 'כ', 1, 0),df.index)                           
    df['worked10'] = pd.Series(np.where(df.worked10.values == 'כ', 1, 0),df.index)                           
    df['worked9'] = pd.Series(np.where(df.worked9.values == 'כ', 1, 0),df.index)                           
    df['worked8'] = pd.Series(np.where(df.worked8.values == 'כ', 1, 0),df.index)                           
    df['worked7'] = pd.Series(np.where(df.worked7.values == 'כ', 1, 0),df.index)                           
    df['worked6'] = pd.Series(np.where(df.worked6.values == 'כ', 1, 0),df.index)                           
    df['worked5'] = pd.Series(np.where(df.worked5.values == 'כ', 1, 0),df.index)                           
    df['worked4'] = pd.Series(np.where(df.worked4.values == 'כ', 1, 0),df.index)                           
    df['worked3'] = pd.Series(np.where(df.worked3.values == 'כ', 1, 0),df.index)                           
    df['worked2'] = pd.Series(np.where(df.worked2.values == 'כ', 1, 0),df.index)                           
    df['worked1'] = pd.Series(np.where(df.worked1.values == 'כ', 1, 0),df.index)                           


    df['status'] = pd.Series(np.where(df.status.values == 'עובד      ', 'employed', np.where(df.status.values == 'עצמאי     ','freelance',np.nan)),df.index)
    df['report'] = pd.Series(np.where(df.report.values == 'ס', 'final',np.where(df.report.values == 'ש','yearly',
                                                        np.where(df.report.values == 'ח','monthly',np.nan))),df.index)
    # l = list(df['wage'])
    df = df.sort_values(['identified'],ascending = [False]).drop_duplicates(subset = ['ID','year_of_wage','employer_name']
                                                                           ).reset_index().drop(['index'], axis = 1)
    

    def int_no_minus(n):
        try:
            return int(n)
        except:
            return 0


    df['wage'] = df['wage'].apply(int_no_minus)

    df['DB'] = pd.to_datetime(df['date_of_birth'],errors = 'coerce')
    df['count_worked'] = df.worked1+df.worked2+df.worked3+df.worked4+df.worked5+df.worked6+df.worked7+df.worked8+df.worked9+df.worked10+df.worked11+df.worked12
    df.loc[(df['count_worked'] == 0) & (df['wage'] > 0),['worked'+str(i) for i in range(1,13)]] = 1
    df = df.drop_duplicates()


    return df

#### 5. Mapping

In [ ]:
colleges1 = pd.read_excel(r'college_data.xlsx', sheet_name='Students')
israel = mpimg.imread('israel-vector-country-map-outline-2JFP21J.jpg')

def map_from_image(colleges1,israel):
    fig, ax = plt.subplots(figsize=(9, 8))
    ax.imshow(israel, extent=[32.37, 37.72, 28.8, 33.75])

    ax.scatter(colleges1['אורך'], colleges1['רוחב'], color='gray', alpha=0.7)

    ax.set_xlim([32.8, 37])
    ax.set_ylim([27.5, 33.6])

    plt.show()

In [ ]:
map_path_high = "ne_10m_admin_0_countries_isr.shp"
colors = ["red", "yellow", "green"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
norm = Normalize(vmin=-1, vmax=1)

def changes_df(df, var):
    df1 = df.groupby(by=['רשות', 'סקר'], as_index=False)[var].mean()
    
    years = [2021, 2022, 2023]
    d = {}
    for rashut in df['רשות'].unique():
        drashut = df[df['רשות'] == rashut]
        
        rashut_means = []
        diffs = []
        for year in years:
            mean_value = drashut[drashut['סקר'] == year][var].mean()
            rashut_means.append(mean_value)
        
        for i in range(1, 3):
            diff = round((rashut_means[i] - rashut_means[i-1]) / rashut_means[i-1], 2)
            diffs.append(diff)
        
        d[rashut] = diffs
    
    dchanges = pd.DataFrame(d).transpose()
    dchanges.columns = ['שיעור שינוי 21-22', 'שיעור שינוי 22-23']
    dchanges['רשות'] = dchanges.index
    
    dchanges = dchanges.reset_index(drop=True)
    dchanges = dchanges[['רשות', 'שיעור שינוי 21-22', 'שיעור שינוי 22-23']]

    return dchanges

def map_by_grade(df, var):
    colors = ["red", "yellow", "green"]
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    norm = Normalize(vmin=-1, vmax=1)
    
    latlng = pd.read_excel(r"C:\Users\SaarPardo\Box\צוות תעסוקה\סוכנות לעסקים קטנים\סקרים\מדד ידידות רשויות מקומיות\2024\פאנל שביעות רצון - מחקר\latlong rashuyot.xlsx")
    latlng = latlng[['רשות', 'אורך', 'רוחב']]

    df1 = changes_df(df, var)
    df1 = df1.merge(latlng, how='left', on='רשות')

    columns = ['שיעור שינוי 21-22', 'שיעור שינוי 22-23']
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    for i, column in enumerate(columns):
        gdf = gpd.GeoDataFrame(df1, geometry=gpd.points_from_xy(df1['רוחב'], df1['אורך']))
        world = gpd.read_file(map_path_high)
        
        world.plot(ax=axes[i], color='linen', edgecolor='white')
        
        gdf.plot(ax=axes[i], markersize = abs(df1[column]) * 80,
                 alpha=0.9, 
                 column=column, 
                 cmap=cmap, 
                 norm=norm)
        
        for _, row in gdf.iterrows():
            if abs(row[column]) > 0.5:
                # Adjust placement based on index to alternate positions and reduce overlap
                if _ % 2 == 0:
                    x_offset = 0.02
                    y_offset = 0.01
                    ha = 'left'
                else:
                    x_offset = -0.02
                    y_offset = -0.01
                    ha = 'right'
                
                axes[i].text(row['רוחב'] + x_offset, row['אורך'] + y_offset, row['רשות'][::-1],
                             fontsize=(7 + (abs(row[column]))**2),  # Slightly reduce the font size scaling
                             ha=ha, 
                             va='bottom',  
                             color='black', 
                             bbox=dict(facecolor='none', edgecolor='none', boxstyle='round,pad=0.3'))
                
        axes[i].set_ylim([31, 33.5])  # Set y-axis limits
        axes[i].set_xlim([34.2, 35.9])  # Set x-axis limits
        axes[i].set_title(f'{column[12::]}:{column[11::-1]} - {var[::-1]}' , fontsize=10, pad=10)
        axes[i].set_xticks([])
        axes[i].set_yticks([])
    
    plt.tight_layout()
    plt.show()

In [ ]:
def markersize(df, mins=5, maxs=100):
    if df['meonot'].max() == df['meonot'].min():
        df['marksize'] = mins
    else:
        min_size = mins  
        max_size = maxs  
        sizes = ((df['meonot'] - df['meonot'].min()) / 
                (df['meonot'].max() - df['meonot'].min()) * 
                (max_size - min_size) + min_size)
        df['marksize'] = sizes
    
    return df

def map_labels_func(df, var, minf=2, maxf=10):
    df_unique = df.sort_values('marksize', ascending=False).drop_duplicates(subset='name', keep='first')
    
    min_lng = df_unique['longitude'].min() - 0.08
    max_lng = df_unique['longitude'].max() + 0.08
    min_lat = df_unique['latitude'].min() - 0.08
    max_lat = df_unique['latitude'].max() + 0.08
    
    gdf = gpd.GeoDataFrame(df_unique, geometry=gpd.points_from_xy(df_unique.longitude, df_unique.latitude))
    world = gpd.read_file("C:/Users/SaarPardo/Desktop/מיפוי חדש משפחתונים/isr mapping/ne_10m_admin_0_countries_isr.shp")
    
    fig, ax = plt.subplots(figsize=(20, 10))
    world.plot(ax=ax, color=(250/255, 250/255, 250/255), edgecolor=(230/255, 230/255, 230/255))
    
    unique_values = df_unique[var].unique()
    colors = {unique_values[0]: 'teal', unique_values[1]: 'orange'}
    gdf.plot(ax=ax, column=var, color=gdf[var].map(colors), legend=True, 
             markersize=df_unique['marksize'], alpha=0.35)
    
    font_sizes = ((df_unique['marksize'] - df_unique['marksize'].min()) / 
                 (df_unique['marksize'].max() - df_unique['marksize'].min()) * 
                 (maxf - minf) + minf)
    
    texts = []
    df_sorted = df_unique.sort_values('marksize', ascending=False)
    direction_flags = [(-0.01, 0), (0.01, 0), (0, -0.01), (0, 0.01)]  # Up, Down, Left, Right
    direction_idx = 0
    
    for idx, row in df_sorted.iterrows():
        label = f"{row['total']} :{row['name'][::-1]}"
        offset_x, offset_y = direction_flags[direction_idx]
        direction_idx = (direction_idx + 1) % len(direction_flags)  # Alternate direction
        
        text = ax.text(row['longitude'] + offset_x * 0.5, 
                       row['latitude'] + offset_y * 0.5, 
                       label, fontsize=font_sizes[idx], alpha=0.8,
                       ha='center', va='center')
        texts.append(text)

    legend_elements = [
        Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=15, label=unique_values[1][::-1]),
        Line2D([0], [0], marker='o', color='w', markerfacecolor='teal', markersize=15, label=unique_values[0][::-1])
    ]
    ax.legend(handles=legend_elements, loc='upper left', fontsize=15)
    ax.set_xlim([min_lng, max_lng])
    ax.set_ylim([min_lat, max_lat])
    plt.show()
    return fig

map = map_labels_func(markersize(meonot,mins=20,maxs=400),'type',
                     minf=3 , maxf=10)
map.savefig("map4.pdf", format="pdf", dpi=150, bbox_inches="tight")

#### 6. Factor analysis

In [ ]:
pca_df=pca_df.dropna()

fa = FactorAnalyzer(n_factors=19, rotation=None)
fa.fit(pca_df)
ev, v = fa.get_eigenvalues()
ev

plt.scatter(range(1,pca_df.shape[1]+1),ev)
plt.plot(range(1,pca_df.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

In [ ]:
fa = FactorAnalyzer(n_factors=3, rotation='varimax')
fa.fit(pca_df)

factor_loadings = fa.loadings_
loadings_df = pd.DataFrame(factor_loadings, index=pca_df.columns, columns=['Factor 1', 'Factor 2', 'Factor 3'])
loadings_df

#### 7. Cornbach's Alpha (Manual calculation)

In [ ]:
def cronbach_alpha(data):
    # Calculate number of items (N)
    n_items = data.shape[1]
    
    # Calculate item variances (v̄)
    item_variances = data.var(axis=0, ddof=1)
    
    # Calculate average inter-item covariance (c̄)
    # First get covariance matrix
    covariance_matrix = data.cov()
    
    # Calculate average covariance (excluding diagonal variances)
    n = covariance_matrix.shape[0]
    # Use numpy to get diagonal
    covariance_sum = (covariance_matrix.sum().sum() - np.diag(covariance_matrix).sum())
    avg_covariance = covariance_sum / (n * (n - 1))
    
    # Apply Cronbach's alpha formula: α = Nc̄ / (v̄ + (N-1)c̄)
    avg_variance = item_variances.mean()
    alpha = (n_items * avg_covariance) / (avg_variance + (n_items - 1) * avg_covariance)
    
    # Since your code expects two return values, let's return alpha and None
    return alpha, None

#### 8. Entorpy Blancing - [Hainmueller (2012)]

In [ ]:
from pyeb import entbal

mydata["T"] = (mydata["sug_seker"] == "טלפוני").astype(int)

X = pd.get_dummies(mydata[["size", "sector", "A1"]], drop_first=False)

# התאמה לפי ATT (איזון הביקורת לפי הטיפול)
eb = entbal()
eb.fit(X, mydata["T"], estimand="ATT")

mydata["weight"] = eb.W

In [ ]:
mydata_needed = mydata[mydata["A1"] == "זקוק למימון"].copy()

mydata_needed["T"] = (mydata_needed["sug_seker"] == "טלפוני").astype(int)

X1 = pd.get_dummies(mydata_needed[["size", "sector"]], drop_first=False)

# התאמה לפי ATT (איזון הביקורת לפי הטיפול)
eb = entbal()
eb.fit(X1, mydata_needed["T"], estimand="ATT")

mydata_needed["en_weight"] = eb.W 

mydata_needed["classic_w"] = np.where(mydata_needed["en_weight"] == 1, mydata_needed["en_weight"], 
                                mydata_needed["en_weight"]*len(mydata_needed[mydata_needed['sug_seker']=='אינטרנטי']))